# Chapter5. 구조적 API 기본 연산

DataFrame의 기본 기능을 중점으로 DataFrame의 데이터를 다루는 기능 소개. 

DataFrame은 Row 타입의 레코드와 각 레코드에 수행할 연산 표현식을 나타내는 여러 컬럼으로 구성.

* 스키마 : 각 컬럼명과 데이터 타입 정의
* 파티셔닝 : DataFrame이나 Dataset이 클러스터에서 물리적으로 배치되는 형태 정의
* 파티셔닝 스키마 : 파티션을 배치하는 방법 정의
* 파티셔닝의 분할 기준은 특정 컬럼이나 비결정론적 값을 기반으로 설정

DataFrame 생성

In [0]:
df = spark.read.format("json").load("dbfs:/FileStore/tables/data/2015_summary.json")

DataFrame의 스키마

In [0]:
df.printSchema()

root
-- DEST_COUNTRY_NAME: string (nullable = true)
-- ORIGIN_COUNTRY_NAME: string (nullable = true)
-- count: long (nullable = true)

## 5.1 스키마

스키마? DataFrame의 컬럼명과 데이터 타입 지ㅓㅇ의. 

예제 데이터 : 미국 교통통계국이 제공하는 항공운항 데이터이며 줄로 구분된 반정형 JSON 데이터

In [0]:
spark.read.format("json").load("dbfs:/FileStore/tables/data/2015_summary.json").schema

Out[4]: StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

스키마는 여러 개의 StructField 타입 필드로 구성된 StructType 객체. 

* StructField : 이름, 데이터 타입, 컬럼이 값이 없거나 null일 수 있는지 지정하는 불리언 값을 가짐. 
* 메타데이터 : 해당 컬럼과 관련된 정보이며 스파크의 머신러닝 라이브러리에서 사용.
* 스키마는 복합 데이터 타입인 StructType을 가질 수 있음.
* 스파크는 런타임에 데이터 타입이 스키마의 데이터 타입과 일치하지 않으면 오류 발생.

예제) DataFrame에 스키마를 만들고 적용

In [0]:
from pyspark.sql.types import StructField, StructType, StringType, LongType

myManualSchema = StructType([
  StructField("DEST_COUNTRY_NAME", StringType(), True),
  StructField("ORIGIN_COUNTRY_NAME", StringType(), True),
  StructField("count", LongType(), False, metadata={"hello":"world"})
])

df = spark.read.format("json").schema(myManualSchema)\
  .load("dbfs:/FileStore/tables/data/2015_summary.json")

## 5.2 컬럼과 표현식

스파크의 컬럼은 스프레드시트, R의 dataframe, Pandas의 DataFrame 컬럼과 유사.
* 사용자는 표현식으로 DataFrame의 컬럼을 선택, 조작, 제거 가능
* 스파크의 컬럼은 표현식을 사용해 레코드 단위로 계산한 값을 단순하게 나타내는 논리적인 구조

따라서 컬럼의 실젯값을 얻으려면 로우가 필요하고, 로우를 얻으려면 DataFrame이 필요

* DataFrame을 통하지 않으면 외부에서 컬럼에 접근 불가
* 컬럼 내용을 수정하려면 반드시 DataFrame의 스파크 트랜스포메이션 사용

### 5.2.1 컬럼

col 함수나 column 함수에서 컬럼명을 인수로 받아 컬럼을 생성하고 참조.

In [0]:
from pyspark.sql.functions import col, column

col("someColumnName")
column("someColumnName") # 오류 발생

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-446413481131103> in <module> 
 2 
 3 col ( "someColumnName" ) 
 ----> 4 column ( "someColumnName" ) 

 /databricks/spark/python/pyspark/sql/functions.py in column (col) 
 113 Returns a : class : ` ~ pyspark . sql . Column ` based on the given column name . ' 
 114 """
 --> 115 return col ( col ) 
 116 
 117 

 TypeError : 'str' object is not callable

컬럼이 DataFrame에 있을지 없을지는 알 수 없다. 컬럼은 컬럼명을 카탈로그에 저장된 정보와 비교하기 전까지 미확인 상태로 남기 때문.

##### 명시적 컬럼 참조

DataFrame의 컬럼은 col 메서드로 참조.

* 조인 시 유용 - DataFrame의 어떤 컬럼을 다른 DataFrame의 조인 대상 컬럼에서 참조하기 위해 col 메서드 사용.
* col 메서드를 사용해 명시적으로 컬럼을 정의하면 스파크는 분석기 실행 단계에서 컬럼 확인 절차 생략

### 5.2.2 표현식

표현식? DataFrame 레코드의 여러 값에 대한 트랜스포메이션 집합 의미. 즉, 여러 컬럼명을 입력으로 받아 실별하고, '단일 값'을 만들기 위해 다앙햔 표현식을 각 레코드에 적용하는 함수. 

(단일 값은 Map이나 Array 같은 복합 데이터 타입)

* 표현식은 expr 함수로 사용 - DataFrame의 컬럼 참조 가능. expr("someCol")과 col("someCol")은 동일한 구문

##### 표현식으로 컬럼 표현

컬럼은 표현식의 일부 기능 제공. 

* col() 함수를 호출해 컬럼에 트랜스포메이션을 수행하려면 반드시 컬럼 참조 사용
* expr 함수의 인수로 표현식을 사용하면 표현식을 분석해 트랜스포메이션과 컬럼 참조를 알아낼 수 있고, 다음 트랜스포메이션에 컬럼 참조 전달 가능

##### DataFrame 컬럼에 접근하기

printSchema 메서드로 DataFrame의 전체 컬럼 정보를 확인 가능하지만 프로그래밍 방식으로 컬럼에 접근할 때는 DataFrame의 columns 속성 사용.

In [0]:
spark.read.format("json").load("dbfs:/FileStore/tables/data/2015_summary.json").columns

Out[11]: ['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

## 5.3 레코드와 로우

스파크에서 DataFrame의 각 로우는 하나의 레코드. 
* 스파크는 레코드를 Row 객체로 표현. 
* 스파크는 값을 생성하기 위해 컬럼 표현식으로 Row 객체를 다룸.
* Row 객체는 내부에 바이트 배열을 가지므로 바이트 배열 인터페이스는 오직 컬럼 표현식으로만 다룰 수 있어 사용자에게 노출되지 않는다.

In [0]:
# DataFrame의 first 메서드로 로우 확인
df.first()

Out[12]: Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)

### 5.3.1 로우 생성하기

각 컬럼에 해당하는 값을 사용해 Row 객체 직접 생성 가능. 

* Row 객체는 스키마 정보를 가지고 있지 않는다.
* DataFrame만 유일하게 스키마를 갖는다.

따라서 Row 객체를 직접 생성하려면 DataFrame의 스키마와 같은 순서로 값을 명시해야 함.

In [0]:
from pyspark.sql import Row

myRow = Row("Hello", None, 1, False)

로우의 데이터에 접근하려면 원하는 위치 지정

In [0]:
myRow[0]
myRow[2]

Out[14]: 1

## 5.4 DataFrame의 트랜스포메이션

DataFrame을 다루는 방법의 주요 작엉

* 로우나 컬럼 추가
* 로우나 컬럼 제거
* 로우를 컬럼으로 변환하거나 그 반대로 변환
* 컬럼 값을 기준으로 로우 순서 변경

### 5.4.1 DataFrame 생성하기

(임시 뷰로 등록해 사용)

In [0]:
df = spark.read.format("json").load("dbfs:/FileStore/tables/data/2015_summary.json")
df.createOrReplaceTempView("dfTable")

Row 객체를 가진 Seq 타입을 직접 변환해 DataFrame을 생성하는 것도 가능

유용하게 사용하는 메서드
* select 메서드 : 컬럼이나 표현식을 사용
* selectExpr 메서드 : 문자열 표현식 사용
* 메서드로 사용할 수 없는 org.apache.spark.sql.functions 패키지에 포함된 다양한 함수

### 5.4.2 select과 selectExpr

In [0]:
df.select("DEST_COUNTRY_NAME").show(2)

+-----------------+
DEST_COUNTRY_NAME|
+-----------------+
 United States|
 United States|
+-----------------+
only showing top 2 rows

여러 컬럼을 선택하려면 select 메서드에 원하는 컬럼명 추가

In [0]:
df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(2)

+-----------------+-------------------+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
 United States| Romania|
 United States| Croatia|
+-----------------+-------------------+
only showing top 2 rows

다양한 컬럼 참조 방법

In [0]:
from pyspark.sql.functions import expr, col, column

df.select(
  expr("DEST_COUNTRY_NAME"),
  col("DEST_COUNTRY_NAME"),
  column("DEST_COUNTRY_NAME"))\  # Column 객체와 문자열을 함께 쓰면 오류 발생.
  .show(2)

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-446413481131123> in <module> 
 4 expr ( "DEST_COUNTRY_NAME" ) , 
 5 col ( "DEST_COUNTRY_NAME" ) , 
 ----> 6 column("DEST_COUNTRY_NAME"))\
 7 . show ( 2 ) 

 /databricks/spark/python/pyspark/sql/functions.py in column (col) 
 113 Returns a : class : ` ~ pyspark . sql . Column ` based on the given column name . ' 
 114 """
 --> 115 return col ( col ) 
 116 
 117 

 TypeError : 'str' object is not callable

expr 함수는 단순 컬럼 참조나 문자열을 이용해 컬럼명 참조 가능

AS 키워드로 컬럼명을 변경한 다음 alias 메서드로 원래 컬럼명으로 되돌리는 코드 작성

In [0]:
df.select(expr("DEST_COUNTRY_NAME AS destination")).show(2)

+-------------+
 destination|
+-------------+
United States|
United States|
+-------------+
only showing top 2 rows

표현식의 결과를 다른 표현식으로 다시 처리 가능

In [0]:
df.select(expr("DEST_COUNTRY_NAME as destination").alias("DEST_COUNTRY_NAME"))\
  .show(2)

+-----------------+
DEST_COUNTRY_NAME|
+-----------------+
 United States|
 United States|
+-----------------+
only showing top 2 rows

selectExpr 메서드 : select 메서드에 expr 함수를 사용하는 패턴을 효율적으로 작성할 수 있도록 함.

In [0]:
df.selectExpr("DEST_COUNTRY_NAME as newColumnName", "DEST_COUNTRY_NAME").show(2)

+-------------+-----------------+
newColumnName|DEST_COUNTRY_NAME|
+-------------+-----------------+
United States| United States|
United States| United States|
+-------------+-----------------+
only showing top 2 rows

selectExpr 메서드는 새로운 DataFrame을 생성하는 복잡한 표현식을 간단하게 만드는 도구. 

예제) DataFrame에 출발지와 도착지가 같은지 나타내는 새로운 withinCountry 컬럼 추가

In [0]:
df.selectExpr(
  "*",
  "(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as whithinCountry")\
  .show(2)

+-----------------+-------------------+-----+--------------+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|whithinCountry|
+-----------------+-------------------+-----+--------------+
 United States| Romania| 15| false|
 United States| Croatia| 1| false|
+-----------------+-------------------+-----+--------------+
only showing top 2 rows

select 표현식에는 DataFrame의 컬럼에 대한 집계 함수 지정 가능

In [0]:
df.selectExpr("avg(count)", "count(distinct(DEST_COUNTRY_NAME))").show(2)

+-----------+---------------------------------+
 avg(count)|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
1770.765625| 132|
+-----------+---------------------------------+

### 5.4.3 스파크 데이터 타입으로 변환하기

새로운 컬럼이 아닌 명시적인 값을 스파크에 전달할 때 리터럴 사용. 

리터럴은 프로그래밍 언어의 리터럴 값을 스파크가 이해할 수 있는 값으로 변환.

어떤 상수나 프로그래밍으로 생성된 변숫값이 특정 컬럼의 값보다 큰지 확인할 때 주로 사용.

In [0]:
from pyspark.sql.functions import lit

df.select(expr("*"), lit(1).alias("One")).show(2)

+-----------------+-------------------+-----+---+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
 United States| Romania| 15| 1|
 United States| Croatia| 1| 1|
+-----------------+-------------------+-----+---+
only showing top 2 rows

### 5.4.4 컬럼 추가하기

DataFrame의 withColumn 메서드를 사용해 신규 컬럼 추가 가능.

예제) 숫자 1을 값으로 가지는 컬럼 추가

In [0]:
df.withColumn("numberOne", lit(1)).show(2)

+-----------------+-------------------+-----+---------+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
 United States| Romania| 15| 1|
 United States| Croatia| 1| 1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows

예제) 출발지와 도착지가 같은지 여부를 불리언 타입으로 표현

In [0]:
df.withColumn("withinCountry", expr("ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME")).show(2)

+-----------------+-------------------+-----+-------------+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
 United States| Romania| 15| false|
 United States| Croatia| 1| false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows

withColumn 메서드는 두 개의 인수 컬럼명과 표현식 사용. 

withColumn 메서드로 컬럼명 변경도 가능

In [0]:
df.withColumn("Destination", expr("DEST_COUNTRY_NAME")).columns

Out[27]: ['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count', 'Destination']

### 5.4.5 컬럼명 변경하기

withColumn 메서드 대신 withColumnRenamed 메서드로 첫 번째 인수로 전달된 컬럼명을 두 번째 인수의 문자열로 컬럼명 변경 가능.

In [0]:
df.withColumnRenamed("DEST_COUNTRY_NAME", "dest").columns

Out[28]: ['dest', 'ORIGIN_COUNTRY_NAME', 'count']

### 5.4.6 예약 문자와 키워드

공백이나 하이픈(-) 같은 예약 문자는 컬럼명에 사용할 수 없다. 예약 문자를 컬럼명에 사용하려면 백틱( ` ) 문자를 이용해 이스케이핑 해야 한다.

예제) withColumn 메서드를 사용해 예약 문자가 포함된 컬럼 생성

In [0]:
dfWithLongColName = df.withColumn(
  "This Long Column-Name",
  expr("ORIGIN_COUNTRY_NAME"))

표현식으로 컬럼을 참조하므로 백틱 문자 사용

In [0]:
dfWithLongColName.selectExpr(
  "`This Long Column-Name`",
  "`This Long Column-Name` as `new col1`")\
.show(2)

dfWithLongColName.createOrReplaceTempView("dfTableLong")

+---------------------+--------+
This Long Column-Name|new col1|
+---------------------+--------+
 Romania| Romania|
 Croatia| Croatia|
+---------------------+--------+
only showing top 2 rows

### 5.4.7 대소문자 구분

스파크는 기본적으로 대소문자를 가리지 않지만 다음과 같은 설정을 통해 대소문자를 구분하게 할 수 있다.

-- SQL

set spark.sql.caseSensitive true

### 5.4.8 컬럼 제거하기

DataFrame에서 컬럼을 제거한느 방법 : drop 메서드 사용

In [0]:
df.drop("ORIGIN_COUNTRY_NAME").columns

Out[33]: ['DEST_COUNTRY_NAME', 'count']

In [0]:
# 다수의 컬럼명을 한꺼번에 제거

dfWithLongColName.drop("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME")

Out[34]: DataFrame[count: bigint, This Long Column-Name: string]

### 5.4.9 컬럼의 데이터 타입 변경하기

다수의 StringType 컬럼을 정수형으로 변환하는 경우 cast 메서드 사용

예제) count 컬럼을 Integer 데이터 타입에서 String 데이터 타입으로 형변환

In [0]:
df.withColumn("count2", col("count").cast("string"))

Out[35]: DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint, count2: string]

### 5.4.10 로우 필터링하기

로우를 필터링하려면 참과 거짓을 판별하는 표현식을 만들어야 한다. 그러면 표현식의 결과가 false인 로우를 걸러낼 수 있다.

DataFrame의 where 메서드나 filter 메서드로 필터링 가능.

In [0]:
df.filter(col("count") < 2).show(2)

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 United States| Croatia| 1|
 United States| Singapore| 1|
+-----------------+-------------------+-----+
only showing top 2 rows

In [0]:
df.where("count < 2").show(2)

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 United States| Croatia| 1|
 United States| Singapore| 1|
+-----------------+-------------------+-----+
only showing top 2 rows

여러 개의 AND 필터를 지정하려면 차례대로 필터를 연결

단, 스파크는 자동으로 필터의 순서와 상관없이 동시에 모든 필터링 작업을 수행해 유용하지 않음.

In [0]:
df.where(col("count") < 2).where(col("ORIGIN_COUNTRY_NAME") != "Croatia")\
  .show(2)

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 United States| Singapore| 1|
 Moldova| United States| 1|
+-----------------+-------------------+-----+
only showing top 2 rows

### 5.4.11 고유한 로우 얻기 

고윳값을 얻으려면 하나 이상의 컬럼 사용. 

DataFrame의 모든 로우에서 중복 데이털르 제거할 수 있는 distinct 메서드를 사용해 고윳값을 찾을 수 있다. 

예제) 항송운항 데이터셋에서 중복되지 않은 출발지 정보를 얻기

* distinct 메서드 : 중복되지 않은 로우를 가진 신규 DataFrame 반환

In [0]:
df.select("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").distinct().count()

Out[39]: 256

In [0]:
df.select("ORIGIN_COUNTRY_NAME").distinct().count()

Out[40]: 125

### 5.4.12 무작위 샘플 만들기

DataFrame의 sample 메서드를 사용해 무작위 샘플 생성 가능.

DataFrame에서 표본 데이터 추출 비율을 지정할 수 있으며, 복원 추출과 비복원 추출의 사용 여부도 지정 가능.

In [0]:
seed = 5
withReplacement = False
fraction = 0.5
df.sample(withReplacement, fraction, seed).count()

Out[41]: 138

### 5.4.13 임의 분할하기

임의 분할 : 원본 DataFrame을 임의 크기로 분할할 때 유용하게 사용. 

주로 머신러닝 알고리즘에서 사용할 학습셋, 검증셋, 테스트셋을 만들 때 주로 사용

예제) 분할 가중치를 함수의 파라미터로 설정해 원본 DataFrame을 서로 다른 데이터를 가진 두 개의 DataFrame으로 나눈다.

단, 메서드는 임의성을 가지고 설계되어있어 시드 값을 반드시 설정해야 한다.

In [0]:
dataFrames = df.randomSplit([0.25, 0.75], seed)
dataFrames[0].count() > dataFrames[1].count()

Out[42]: False

### 5.4.14 로우 합치기와 추가하기

DataFrame은 불변성을 가지므로 레코드를 추가하는 작업은 불가능하다. 따라서 새로운 레코드를 추가하려면 원본 DataFrame을 새로운 DataFrame과 통합해야 한다.

통합은 단순히 결합하는 행위로, 통합하려는 두 개의 DataFrame은 반드시 동일한 스키마와 컬럼 수를 가져야 한다.

In [0]:
from pyspark.sql import Row

schema = df.schema
newRows = [
  Row("New Country", "Other Country", 5),
  Row("New Country", "Other Country 3", 1)
]

parallelizedRows = spark.sparkContext.parallelize(newRows)
newDF = spark.createDataFrame(parallelizedRows, schema)

In [0]:
df.union(newDF)\
  .where("count = 1")\
  .where(col("ORIGIN_COUNTRY_NAME") != "United States")\
  .show()

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 United States| Croatia| 1|
 United States| Singapore| 1|
 United States| Gibraltar| 1|
 United States| Cyprus| 1|
 United States| Estonia| 1|
 United States| Lithuania| 1|
 United States| Bulgaria| 1|
 United States| Georgia| 1|
 United States| Bahrain| 1|
 United States| Papua New Guinea| 1|
 United States| Montenegro| 1|
 United States| Namibia| 1|
 New Country| Other Country 3| 1|
+-----------------+-------------------+-----+

### 5.4.15 로우 정렬하기

sort와 orderBy 메서드를 사용해 최댓값 혹은 최솟값이 상단에 위치하도록 정렬 가능. (기본 방식은 오름차순)

In [0]:
df.sort("count").show(5)

+--------------------+-------------------+-----+
 DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
 Malta| United States| 1|
Saint Vincent and...| United States| 1|
 United States| Croatia| 1|
 United States| Gibraltar| 1|
 United States| Singapore| 1|
+--------------------+-------------------+-----+
only showing top 5 rows

In [0]:
df.orderBy("count", "DEST_COUNTRY_NAME").show(5)

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 Burkina Faso| United States| 1|
 Cote d'Ivoire| United States| 1|
 Cyprus| United States| 1|
 Djibouti| United States| 1|
 Indonesia| United States| 1|
+-----------------+-------------------+-----+
only showing top 5 rows

In [0]:
df.orderBy(col("count"), col("DEST_COUNTRY_NAME")).show(5)

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 Burkina Faso| United States| 1|
 Cote d'Ivoire| United States| 1|
 Cyprus| United States| 1|
 Djibouti| United States| 1|
 Indonesia| United States| 1|
+-----------------+-------------------+-----+
only showing top 5 rows

정렬 기준을 명확히 지정하려면 asc나 desc 함수 사용

In [0]:
from pyspark.sql.functions import desc, asc

df.orderBy(expr("count desc")).show(2)

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 Moldova| United States| 1|
 United States| Croatia| 1|
+-----------------+-------------------+-----+
only showing top 2 rows

In [0]:
df.orderBy(col("count").desc(), col("DEST_COUNTRY_NAME").asc()).show(2)

+-----------------+-------------------+------+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
 United States| United States|370002|
 United States| Canada| 8483|
+-----------------+-------------------+------+
only showing top 2 rows

asc_nulls_first, desc_nulls_first, asc_nulls_last, desc_nulls_last 메서드를 사용해 null 값이 표시되는 기준 지정 가능

트랜스포메이션을 처리하기 전에 성능을 최적화하기 위해 파티션별 정렬을 수행하기도 하는데, 파티션별 정렬은 sortWithinPartitions 메서드 사용

### 5.4.16 로우 수 제한하기

limit 메서드를 사용해 추출할 로우 수를 제한할 수 있다.

In [0]:
df.limit(5).show()

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 United States| Romania| 15|
 United States| Croatia| 1|
 United States| Ireland| 344|
 Egypt| United States| 15|
 United States| India| 62|
+-----------------+-------------------+-----+

In [0]:
df.orderBy(expr("count desc")).limit(6).show()

+--------------------+-------------------+-----+
 DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
 Malta| United States| 1|
Saint Vincent and...| United States| 1|
 United States| Croatia| 1|
 United States| Gibraltar| 1|
 United States| Singapore| 1|
 Moldova| United States| 1|
+--------------------+-------------------+-----+

### 5.4.17 repartition과 coalesce

다른 최적화 기법은 자주 필터링 하는 컬럼을 기준으로 데이터를 분할하는 것. 이를 통해 파티셔닝 스키마와 파티션 수를 포함해 클러스터 전반의 물리적 데이터 구성 제어 가능.

repartition 메서드를 호출하면 무조건 전체 데이터 셔플.

In [0]:
df.rdd.getNumPartitions()

Out[55]: 1

In [0]:
df.repartition(5)

Out[56]: DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

특정 컬럼을 기준으로 자주 필터링 한다면 자주 필터링하는 컬럼을 기준으로 파티션을 재분배하는 것이 좋음.

In [0]:
df.repartition(col("DEST_COUNTRY_NAME"))

Out[57]: DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

선택적으로 파티션 수 지정 가능

In [0]:
df.repartition(5, col("DEST_COUNTRY_NAME"))

Out[58]: DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

coalsece 메서드는 전체 데이터를 셔플하지 않고 파티션을 병합하는 경우에 사용

예제) 목적지를 기준으로 셔플을 수행해 5개의 파티션으로 나누고, 전체 데이터를 셔플 없이 병합

In [0]:
df.repartition(5, col("DEST_COUNTRY_NAME")).coalesce(2)

Out[60]: DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

### 5.4.18 드라이버로 로우 데이터 수집하기

스파크는 드라이버에서 클러스터 상태 정보를 유지하기 때문에 로컬 환경에서 데이터를 다루려면 드라이버로 데이터를 수집해야 한다. 
* collect 메서드 : 전체 DataFrame의 모든 데이터 수집
* take 메서드 : 상위 N개의 로우 반환
* show 메서드 : 여러 로우를 보기 좋게 출력

In [0]:
collectDF = df.limit(10)
collectDF.take(5) # take는 정수형 값을 인수로 사용
collectDF.show() # 결과를 정돈된 형태로 출력
collectDF.show(5, False)
collectDF.collect()

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 United States| Romania| 15|
 United States| Croatia| 1|
 United States| Ireland| 344|
 Egypt| United States| 15|
 United States| India| 62|
 United States| Singapore| 1|
 United States| Grenada| 62|
 Costa Rica| United States| 588|
 Senegal| United States| 40|
 Moldova| United States| 1|
+-----------------+-------------------+-----+

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
United States |Romania |15 |
United States |Croatia |1 |
United States |Ireland |344 |
Egypt |United States |15 |
United States |India |62 |
+-----------------+-------------------+-----+
only showing top 5 rows

Out[61]: [Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Grenada', count=62),
 Row(DEST_COUNTRY_NAME='Costa Rica', ORIGIN_COUNTRY_NAME='United States', count=588),
 Row(DEST_COUNTRY_NAME='Senegal', ORIGIN_COUNTRY_NAME='United States', count=40),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

toLocalIterator 메서드는 반복자로 모든 파티션의 데이터를 드라이버에 전달. 

* toLocalIterator 메서드 : 데이터셋의 파티션을 차례로 반복 처리 가능

In [0]:
collectDF.toLocalIterator()

Out[62]: <generator object _local_iterator_from_socket.<locals>.PyLocalIterable.__iter__ at 0x7f1cd991ac10>